In [20]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [21]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']
collection = db['tweets']

Connected successfully!!!


In [22]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'some-tweets', 'tweets', 'tweet_users']

In [23]:
def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """
    
    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']
    return df

In [24]:
df = read_mongo(db, "tweets")

In [25]:
len(df)

10490

In [26]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [27]:
output = author_dataset_conversion(df)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [28]:
output

,j_user,RT_l10,sd_RT,FC_l10,sd_FC,FC,RT
0,"{'id': 2107671, 'id_str': '2107671', 'name': '...",0.4,0.489898,1.7,3.796051,0.0,0.0
1,"{'id': 4869021, 'id_str': '4869021', 'name': '...",8.4,16.805951,76.1,127.668673,25.0,3.0
2,"{'id': 14206520, 'id_str': '14206520', 'name':...",0.2,0.400000,2.5,3.442383,0.0,0.0
3,"{'id': 22317044, 'id_str': '22317044', 'name':...",0.0,0.000000,0.3,0.458258,0.0,0.0
4,"{'id': 22935720, 'id_str': '22935720', 'name':...",3.6,6.711185,2.8,5.015974,0.0,1.0
5,"{'id': 32406749, 'id_str': '32406749', 'name':...",0.1,0.300000,0.7,0.781025,0.0,0.0
6,"{'id': 42402745, 'id_str': '42402745', 'name':...",0.1,0.300000,0.5,0.670820,0.0,0.0
7,"{'id': 49673609, 'id_str': '49673609', 'name':...",0.8,1.249000,4.5,6.917369,1.0,0.0
8,"{'id': 56585060, 'id_str': '56585060', 'name':...",0.0,0.000000,0.0,0.000000,0.0,0.0
9,"{'id': 58451570, 'id_str': '58451570', 'name':...",1.1,1.044031,1.1,1.135782,4.0,3.0


In [26]:
pd.DataFrame.to_csv(output, "C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [71]:
df["user"][0]

{'id': 2664945430,
 'id_str': '2664945430',
 'name': 'Fútbol Club Jumilla',
 'screen_name': 'FCJumilla',
 'location': 'Jumilla, España',
 'description': 'Perfil Oficial del Fútbol Club Jumilla.\nSegunda División B / Grupo IV @RFEF',
 'url': 'https://t.co/VYPxc3mFj6',
 'entities': {'url': {'urls': [{'url': 'https://t.co/VYPxc3mFj6',
     'expanded_url': 'http://www.futbolclubjumilla.es',
     'display_url': 'futbolclubjumilla.es',
     'indices': [0, 23]}]},
  'description': {'urls': []}},
 'protected': False,
 'followers_count': 7162,
 'friends_count': 334,
 'listed_count': 92,
 'created_at': 'Wed Jul 02 19:09:45 +0000 2014',
 'favourites_count': 2804,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 8466,
 'lang': 'es',
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 

In [ ]:
df["user"]

In [72]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in output2["j_user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    return output

In [69]:
output2 = text_dataset(df)

In [70]:
output2

,id,Tweet,j_user,created_at,entities,followers_user,FC,RT
0,1046429145914384386,Desde el #FCJumilla damos nuestra más sincera ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:58:44,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,18,4
1,1044060298443919367,Me estoy volviendo loca hasta por cada una de ...,"{'id': 713359736, 'id_str': '713359736', 'name...",2018-09-24 03:05:47,"{'hashtags': [], 'symbols': [], 'user_mentions...",531,0,0
2,1046424001214648320,🏟 | El #FCJumilla ya está en La Condomina ⚽️...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:38:18,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,4,2
3,1046419267279818752,📹 | El vestuario del #FCJumilla en La Condomin...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:19:29,"{'hashtags': [{'text': 'FCJumilla', 'indices':...",7162,21,6
4,1046417005451128832,📑 Jornada 6 | El colegiado andaluz Guzmán Mans...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:10:30,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,0,0
5,1046416602739208193,🚐 | La expedición ya está lista para poner rum...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 15:08:54,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",7162,10,2
6,1046343505197641728,⏱ | FINAL #FCJumillaB 1-1 @MulenoClubF ⚽️ ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 10:18:26,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,2,0
7,1046323550297239552,✋🏻⏱| DESCANSO #FCJumillaB 0-0 @MulenoClubF ...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 08:59:08,"{'hashtags': [{'text': 'FCJumillaB', 'indices'...",7162,0,0
8,1046300776455458816,🔊 VINÍCOLAS... 🔊 🔉 ¡¡El Filial también juega!...,"{'id': 2664945430, 'id_str': '2664945430', 'na...",2018-09-30 07:28:39,"{'hashtags': [], 'symbols': [], 'user_mentions...",7162,2,1
9,1045426450348888064,#CasadoDimisión,"{'id': 1401745850, 'id_str': '1401745850', 'na...",2018-09-27 21:34:23,"{'hashtags': [{'text': 'CasadoDimisión', 'indi...",329,0,0


In [45]:
a = [tweet["followers_count"] for tweet in output2["j_user"]]

In [29]:
len(output2)

4289

In [30]:
pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [13]:
cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
ids = [user["id"] for user in cleaned["user"]]
cleaned["id_num"] = ids

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
df = cleaned[cleaned["id_num"]==ids[0]].reset_index(drop=True)

In [18]:
df = df[-10:].reset_index(drop=True)

In [19]:
df

,id,created_at,entities,extended_entities,favorite_count,retweet_count,source,text,truncated,user,quoted_status_id,id_num
0,1045965820445241344,Sat Sep 29 09:17:39 +0000 2018,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",NaN,3,0,"<a href=""http://twitter.com/download/android"" ...",📷 | Sesión de entrenamiento en el Miguel Indur...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
1,1045945120531632129,Sat Sep 29 07:55:24 +0000 2018,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",NaN,8,1,"<a href=""http://twitter.com/download/android"" ...","¡Buenos días, vinícolas! ☀️\n\nEl equipo traba...",True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
2,1045761221071392768,Fri Sep 28 19:44:39 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,13,2,"<a href=""http://twitter.com/download/android"" ...",Nuestras condolencias a toda la familia del @c...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
3,1045676746320859137,Fri Sep 28 14:08:58 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,1,0,"<a href=""http://twitter.com/download/android"" ...",⏰ Horario Oficial | Jornada 7\n\n⚽️ @recreofic...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
4,1045676028306370560,Fri Sep 28 14:06:07 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,9,"<a href=""http://twitter.com/download/android"" ...",RT @_Tercer_Tiempo: Buscamos gente que siga la...,False,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
5,1045617559138168832,Fri Sep 28 10:13:47 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,11,2,"<a href=""http://twitter.com/download/android"" ...",PREVIA. Jornada 6: UCAM Murcia CF - FC Jumilla...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
6,1045612865024126979,Fri Sep 28 09:55:08 +0000 2018,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",NaN,2,0,"<a href=""http://twitter.com/download/android"" ...",📷 | Sesión de entrenamiento en @PinatarArena ✅...,True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
7,1045571960904798208,Fri Sep 28 07:12:35 +0000 2018,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",NaN,5,0,"<a href=""http://twitter.com/download/android"" ...","¡Buenos días, vinícolas! ☀️\n\nEl equipo vuelv...",True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
8,1045241143309553664,Thu Sep 27 09:18:02 +0000 2018,"{'hashtags': [{'text': 'VuelvenTusColores', 'i...","{'media': [{'id': 1045241077194731523, 'id_str...",9,1,"<a href=""http://twitter.com/download/android"" ...",📷 | El equipo completa su sesión de entrenamie...,False,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430
9,1045228558774423553,Thu Sep 27 08:28:02 +0000 2018,"{'hashtags': [{'text': 'UCAMMurciaJumilla', 'i...",NaN,8,0,"<a href=""http://twitter.com/download/android"" ...","¡Buenos días, vinícolas! ☀️\n\nEl equipo conti...",True,"{'id': 2664945430, 'id_str': '2664945430', 'na...",NaN,2664945430


In [49]:
a = [1,2,3]
b= [2,4,5]

In [59]:
list(np.array(a)/np.array(b))

[0.5, 0.5, 0.6]